In [1]:
import pandas as pd

In [2]:
from scrapy import Selector

In [3]:
import requests

*****

## Harvest new article urls from web portal's main page

In [4]:
## configuration parameters specific to the Assarih newspaper

#BASE_URL = 'https://www.assarih.com/r4-%D9%85%D8%AC%D8%AA%D9%85%D8%B9'   ##ENCODED FORM OF BASE URL
BASE_URL = 'https://www.assarih.com/r4-مجتمع'
xpath_title = "//div[@class='article']//div[@class='article-title']/a/text()"
xpath_link = "//div[@class='article']//div[@class='article-title']/a/@href"
xpath_date = "//div[@class='article']//div[@class='article-title']/p/text()"
xpath_content = "//div[@class='last-article']//text()"

In [5]:
# for each article extract its (hyperlink, title, date)

res = []
start_time = time.time()

for i in range(1,500):
    url = BASE_URL+'?page={}'.format(i)
    html = requests.get(url).content
    sel = Selector(text = html)
    u = sel.xpath(xpath_title).extract()
    v = sel.xpath(xpath_link).extract()
    w = sel.xpath(xpath_date).extract()
    z = [y for y in [x.strip() for x in w] if y!='']
    if len(u)!=len(v):
        print("warning: u and v are of different lengths.")
    if len(u)!=len(z):
        print("warning: u and z are of different lengths.")
    res = res + list(zip(u,v,z))
    if (i%50)==0:
        print(i)
    #print(i,len(u),len(v),len(z))

print(f"Harvested {len(res)} urls.")
print("--- %s minutes elapsed ---" % ((time.time() - start_time)/60))

50
100
150
200
250
300
350
400
450
Harvested 7485 urls.


****

## Store harvested urls in database

In [10]:
meta_data_df_new = pd.DataFrame(res, columns=['title','hyperlink','date'])
meta_data_df_new.shape

(7485, 3)

In [11]:
meta_data_df_new.head()

,title,hyperlink,date
0,في كافّة الولايات : عصابة تورط حزبا كبيرا في ع...,a161414-في-كافّة-الولايات---عصابة-تورط-حزبا-كب...,24 فيفري 2020
1,تجار ومواطنون يغلقون الطريق المؤدية الى معبر ح...,a161411-تجار-ومواطنون-يغلقون-الطريق-المؤدية-ال...,24 فيفري 2020
2,بنزرت : الإطاحة بعصابة نفذت عملية سطو على منزل,a161399-بنزرت---الإطاحة-بعصابة-نفذت-عملية-سطو-...,24 فيفري 2020
3,في الوردية : يهدد العاملات بسكين ثم يسلبهن,a161396-في-الوردية---يهدد-العاملات-بسكين-ثم-يس...,24 فيفري 2020
4,الإعدام شنقا في حق مغتصب وقاتل طفلة الـ7 سنوات...,a161395-الإعدام-شنقا-في-حق-مغتصب-وقاتل-طفلة-ال...,24 فيفري 2020


### cleanup hyperlink field
convert from relative to absolute url

In [12]:
## remove duplicate urls

meta_data_df_new.drop_duplicates(subset='hyperlink',inplace=True)
meta_data_df_new.shape

(7484, 3)

In [14]:
BASE_URL0 = 'https://www.assarih.com/'

In [15]:
## convert to absolute url

meta_data_df_new['hyperlink'] = meta_data_df_new.hyperlink.map(lambda url: BASE_URL0+url)
meta_data_df_new.head()

,title,hyperlink,date
0,في كافّة الولايات : عصابة تورط حزبا كبيرا في ع...,https://www.assarih.com/a161414-في-كافّة-الولا...,24 فيفري 2020
1,تجار ومواطنون يغلقون الطريق المؤدية الى معبر ح...,https://www.assarih.com/a161411-تجار-ومواطنون-...,24 فيفري 2020
2,بنزرت : الإطاحة بعصابة نفذت عملية سطو على منزل,https://www.assarih.com/a161399-بنزرت---الإطاح...,24 فيفري 2020
3,في الوردية : يهدد العاملات بسكين ثم يسلبهن,https://www.assarih.com/a161396-في-الوردية---ي...,24 فيفري 2020
4,الإعدام شنقا في حق مغتصب وقاتل طفلة الـ7 سنوات...,https://www.assarih.com/a161395-الإعدام-شنقا-ف...,24 فيفري 2020


In [16]:
meta_data_df_new.tail()

,title,hyperlink,date
7480,أليم في صفاقس: ارتفاع عدد التلاميذ ضحايا اغتص...,https://www.assarih.com/a129258--أليم-في-صفاقس...,09 مارس 2019
7481,عوسجة (بنزرت): إيقاف «جزّار» تعمد ذبح بقرة مر...,https://www.assarih.com/a129256--عوسجة--بنزرت-...,09 مارس 2019
7482,داهمت السلطات الالمانية منزله بكولونيا: التو...,https://www.assarih.com/a129255---داهمت-السلطا...,09 مارس 2019
7483,خاص/ في قفصة: غرس سكينا في رأس غريمه اثر جلسة...,https://www.assarih.com/a129254-خاص---في-قفصة-...,09 مارس 2019
7484,فجر اليوم بالقيروان : طلق ناري استهدف فلاحا من...,https://www.assarih.com/a129250-فجر-اليوم-بالق...,09 مارس 2019


### parse date field

In [17]:
import dateparser

**Scratch Pad**

In [18]:
dateparser.parse('12/12/12')

datetime.datetime(2012, 12, 12, 0, 0)

In [19]:
dateparser.parse('2015, Ago 15, 1:08 pm', languages=['pt', 'es'])

datetime.datetime(2015, 8, 15, 13, 8)

In [20]:
x = meta_data_df_new.date.iloc[-1]
print(x)
y = dateparser.parse(x)
type(y)

09 مارس 2019


datetime.datetime

In [21]:
y

datetime.datetime(2019, 3, 9, 0, 0)

In [22]:
y.month,y.day

(3, 9)

In [23]:
pd.to_datetime(y)

Timestamp('2019-03-09 00:00:00')

In [24]:
x = meta_data_df_new.date.iloc[-1000]
print(x)
y = dateparser.parse(x,languages=['ar'])
y==None

29 أفريل 2019


True

**End Scratch Pad**

Remark about method using the `dateparser` library :
- This method failed for many dates and produced None values; for example '29 أفريل 2019'. Some dates were parsed correctly such as '09 مارس 2019'.
- Apparently this is because some month names were not recognized by the dateparser library.
- In fact, month names used in Tunisia --and North Africa-- are Arabic transliterations of French month names, a legacy of French colonialism.
- Most likely the `dateparser` library uses month names used in Arab countries of the Levant and Gulf regions ...

In [25]:
## First method: using the dateparser library
## took a long while to run (several minutes)

# %time meta_data_df_new['date_dt'] = meta_data_df_new.date.map(dateparser.parse)

In [ ]:
## therefore I decided to write my own date parsing method, combined with Pandas's to_datetime method.

tunisian_month_to_number_dict = {'جانفي':1,
 'ديسمبر':12,
 'أكتوبر':10,
 'نوفمبر':11,
 'أوت':8,
 'ماي':5,
 'جويلية':7,
 'سبتمبر':9,
 'فيفري':2,
 'جوان':6,
 'أفريل':4,
 'مارس':3}

df1 = meta_data_df_new.date.str.findall('(\d{1,2})(.*)(20\d\d)').map(lambda x: x[0])
df2 = pd.DataFrame(df1.tolist(),columns=['day','month','year'])
df2['month_nb'] = df2.month.str.strip().map(tunisian_month_to_number_dict)

In [46]:
df2['date_dt'] = df2.apply(lambda row: pd.to_datetime(' '.join([str(row.month_nb),row.day,row.year])), axis=1)
df2.set_index(meta_data_df_new.index,inplace=True)
df2.tail()

,day,month,year,month_nb,date_dt
7480,09,مارس,2019,3,2019-03-09
7481,09,مارس,2019,3,2019-03-09
7482,09,مارس,2019,3,2019-03-09
7483,09,مارس,2019,3,2019-03-09
7484,09,مارس,2019,3,2019-03-09


In [47]:
meta_data_df_new['date_dt'] = df2['date_dt']
meta_data_df_new.head()

,title,hyperlink,date,date_dt
0,في كافّة الولايات : عصابة تورط حزبا كبيرا في ع...,https://www.assarih.com/a161414-في-كافّة-الولا...,24 فيفري 2020,2020-02-24
1,تجار ومواطنون يغلقون الطريق المؤدية الى معبر ح...,https://www.assarih.com/a161411-تجار-ومواطنون-...,24 فيفري 2020,2020-02-24
2,بنزرت : الإطاحة بعصابة نفذت عملية سطو على منزل,https://www.assarih.com/a161399-بنزرت---الإطاح...,24 فيفري 2020,2020-02-24
3,في الوردية : يهدد العاملات بسكين ثم يسلبهن,https://www.assarih.com/a161396-في-الوردية---ي...,24 فيفري 2020,2020-02-24
4,الإعدام شنقا في حق مغتصب وقاتل طفلة الـ7 سنوات...,https://www.assarih.com/a161395-الإعدام-شنقا-ف...,24 فيفري 2020,2020-02-24


In [48]:
meta_data_df_new.tail()

,title,hyperlink,date,date_dt
7480,أليم في صفاقس: ارتفاع عدد التلاميذ ضحايا اغتص...,https://www.assarih.com/a129258--أليم-في-صفاقس...,09 مارس 2019,2019-03-09
7481,عوسجة (بنزرت): إيقاف «جزّار» تعمد ذبح بقرة مر...,https://www.assarih.com/a129256--عوسجة--بنزرت-...,09 مارس 2019,2019-03-09
7482,داهمت السلطات الالمانية منزله بكولونيا: التو...,https://www.assarih.com/a129255---داهمت-السلطا...,09 مارس 2019,2019-03-09
7483,خاص/ في قفصة: غرس سكينا في رأس غريمه اثر جلسة...,https://www.assarih.com/a129254-خاص---في-قفصة-...,09 مارس 2019,2019-03-09
7484,فجر اليوم بالقيروان : طلق ناري استهدف فلاحا من...,https://www.assarih.com/a129250-فجر-اليوم-بالق...,09 مارس 2019,2019-03-09


In [49]:
## double-check that the month and day were not swapped somehow

meta_data_df_new.date_dt.tail().map(lambda x: x.month)

7480    3
7481    3
7482    3
7483    3
7484    3
Name: date_dt, dtype: int64

In [50]:
meta_data_df_new.dtypes

title                object
hyperlink            object
date                 object
date_dt      datetime64[ns]
dtype: object

****

## Download text content from harvested articles' urls

In [51]:
import os.path
#from os import path

In [52]:
data_folder_path = '../data/'
outputfile_base_name = 'www.assarih.com'

In [53]:
meta_data_df_new.shape

(7484, 4)

In [61]:
import re
import time
start_time = time.time()
filenames = []

for i,x in meta_data_df_new.iterrows():
    title,url = x.title,x.hyperlink
    # check if record already exists
    title_ = re.sub('\W','_',title[0:70])
    filename = outputfile_base_name + "__" + title_ + '.txt'
    filenames.append(filename)
    filename = data_folder_path + filename
    if not(os.path.exists(filename)):
        # download raw html page then extract text content
        html = requests.get(url).content
        sel = Selector(text = html)
        u = sel.xpath(xpath_content).extract()
        v = ' '.join([x.strip() for x in u]).strip()
        if len(v)>0:
            # write content to a text file
            with open(filename, 'w', encoding='utf8') as f:
                f.write(url+'\n')
                f.write(v)
    if (i%100)==0:
        print(i)

print("--- %s minutes elapsed ---" % ((time.time() - start_time)/60))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
--- 0.2959282000859578 minutes elapsed ---


In [73]:
assert meta_data_df_new.shape[0]==len(filenames)

In [74]:
meta_data_df_new['filename'] = pd.Series(filenames,index=meta_data_df_new.index)

In [75]:
meta_data_df_new.head()

,title,hyperlink,date,date_dt,filename
0,في كافّة الولايات : عصابة تورط حزبا كبيرا في ع...,https://www.assarih.com/a161414-في-كافّة-الولا...,24 فيفري 2020,2020-02-24,www.assarih.com__في_كاف_ة_الولايات___عصابة_تور...
1,تجار ومواطنون يغلقون الطريق المؤدية الى معبر ح...,https://www.assarih.com/a161411-تجار-ومواطنون-...,24 فيفري 2020,2020-02-24,www.assarih.com__تجار_ومواطنون_يغلقون_الطريق_ا...
2,بنزرت : الإطاحة بعصابة نفذت عملية سطو على منزل,https://www.assarih.com/a161399-بنزرت---الإطاح...,24 فيفري 2020,2020-02-24,www.assarih.com__بنزرت___الإطاحة_بعصابة_نفذت_ع...
3,في الوردية : يهدد العاملات بسكين ثم يسلبهن,https://www.assarih.com/a161396-في-الوردية---ي...,24 فيفري 2020,2020-02-24,www.assarih.com__في_الوردية___يهدد_العاملات_بس...
4,الإعدام شنقا في حق مغتصب وقاتل طفلة الـ7 سنوات...,https://www.assarih.com/a161395-الإعدام-شنقا-ف...,24 فيفري 2020,2020-02-24,www.assarih.com__الإعدام_شنقا_في_حق_مغتصب_وقات...


In [76]:
meta_data_df_new.tail()

,title,hyperlink,date,date_dt,filename
7480,أليم في صفاقس: ارتفاع عدد التلاميذ ضحايا اغتص...,https://www.assarih.com/a129258--أليم-في-صفاقس...,09 مارس 2019,2019-03-09,www.assarih.com___أليم_في_صفاقس__ارتفاع_عدد_ال...
7481,عوسجة (بنزرت): إيقاف «جزّار» تعمد ذبح بقرة مر...,https://www.assarih.com/a129256--عوسجة--بنزرت-...,09 مارس 2019,2019-03-09,www.assarih.com___عوسجة__بنزرت___إيقاف__جز_ار_...
7482,داهمت السلطات الالمانية منزله بكولونيا: التو...,https://www.assarih.com/a129255---داهمت-السلطا...,09 مارس 2019,2019-03-09,www.assarih.com____داهمت_السلطات_الالمانية_منز...
7483,خاص/ في قفصة: غرس سكينا في رأس غريمه اثر جلسة...,https://www.assarih.com/a129254-خاص---في-قفصة-...,09 مارس 2019,2019-03-09,www.assarih.com__خاص___في_قفصة__غرس_سكينا_في_ر...
7484,فجر اليوم بالقيروان : طلق ناري استهدف فلاحا من...,https://www.assarih.com/a129250-فجر-اليوم-بالق...,09 مارس 2019,2019-03-09,www.assarih.com__فجر_اليوم_بالقيروان___طلق_نار...


****

## Store data frame in pickle file

In [65]:
pickle_filepath = '../metadata/articles_meta_data.pickle'
csv_filepath = '../metadata/articles_meta_data.csv'

In [66]:
import pickle

In [78]:
if os.path.exists(pickle_filepath):
    print(pickle_filepath,'already exists.')
    with open(pickle_filepath, 'rb') as f:
        meta_data_df_0 = pickle.dump(f)
        print(meta_data_df_0.shape,'existing records.')
        meta_data_df = pd.concat([meta_data_df_0,meta_data_df_new],axis=0)
        # TO DO: remove duplicates
else:
    meta_data_df = meta_data_df_new

print(meta_data_df.shape,'records in updated database.')

(7484, 5) records in updated database.


In [79]:
with open(pickle_filepath, 'wb') as f:
    pickle.dump(meta_data_df, f)

In [80]:
# meta_data_df.to_csv(csv_filepath,index=True,encoding="utf8")